In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d agungmrf/indonesian-sign-language-bisindo

Dataset URL: https://www.kaggle.com/datasets/agungmrf/indonesian-sign-language-bisindo
License(s): unknown
100% 1.36G/1.36G [00:14<00:00, 85.0MB/s]
100% 1.36G/1.36G [00:14<00:00, 100MB/s] 


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/indonesian-sign-language-bisindo.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
%cd bisindo

/content/bisindo


In [5]:
import os
import shutil
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from PIL import Image
from tensorflow.keras import layers, models


In [6]:
img_width= 224
img_height=224

In [7]:
data_train_path = 'images/train'
data_val_path = 'images/val'

In [8]:
train_datagen = ImageDataGenerator(
                    rescale=1. / 255,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    validation_split=0.2,
                    rotation_range=20
                    )

In [9]:
val_datagen = ImageDataGenerator(
                    rescale=1. / 255,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    validation_split=0.2,
                    rotation_range=20
                    )

In [10]:
train_generator = train_datagen.flow_from_directory(
        data_train_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 9169 images belonging to 26 classes.


In [11]:
validation_generator = val_datagen.flow_from_directory(
        data_val_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 2301 images belonging to 26 classes.


In [12]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(26, activation='softmax')
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
for data_batch, labels_batch in train_generator:
    print("Bentuk data batch:", data_batch.shape)
    print("Bentuk label batch:", labels_batch.shape)
    break


Bentuk data batch: (32, 224, 224, 3)
Bentuk label batch: (32, 26)


In [18]:
history=model.fit(train_generator,
                  validation_data=validation_generator,
                  epochs=20,
                  steps_per_epoch=20,
                  validation_steps=3,
                  verbose=1)

Epoch 1/20
20/20 [==============================] - 88s 4s/step - loss: 1.2034 - accuracy: 0.6969 - val_loss: 1.1019 - val_accuracy: 0.7396
Epoch 2/20
20/20 [==============================] - 87s 4s/step - loss: 1.0548 - accuracy: 0.7094 - val_loss: 1.4207 - val_accuracy: 0.6250
Epoch 3/20
20/20 [==============================] - 86s 4s/step - loss: 1.0280 - accuracy: 0.7297 - val_loss: 1.4153 - val_accuracy: 0.6562
Epoch 4/20
20/20 [==============================] - 87s 4s/step - loss: 1.1014 - accuracy: 0.7281 - val_loss: 1.1282 - val_accuracy: 0.6979
Epoch 5/20
20/20 [==============================] - 87s 4s/step - loss: 1.0989 - accuracy: 0.7219 - val_loss: 1.0440 - val_accuracy: 0.6979
Epoch 6/20
20/20 [==============================] - 86s 4s/step - loss: 0.9490 - accuracy: 0.7504 - val_loss: 1.1371 - val_accuracy: 0.7188
Epoch 7/20
20/20 [==============================] - 87s 4s/step - loss: 0.9734 - accuracy: 0.7422 - val_loss: 0.6383 - val_accuracy: 0.8125
Epoch 8/20
20/20 [==